In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '..')

In [ ]:
# parameter settings
is_plot = False
is_export = False

landmarks_path = '../data/landmarks/6kmh_braless_18markers_12fps.pkl'
meshes_path = '../data/meshes/6kmh_braless_26markers/'
test_landmarks_path = '../data/test/braless_random_landmarks.pkl'

start=0
stride = 12
end=120

# Data Loading

In [ ]:
from mesh4d import obj3d

mesh_ls, texture_ls = obj3d.load_mesh_series(
    folder=meshes_path,
    start=start,
    stride=stride,
    end=end,
)

In [ ]:
from mesh4d import utils

landmarks = utils.load_pkl_object(landmarks_path)
landmarks.interp_field()

In [ ]:
from mesh4d.analyse.crave import clip_with_contour

contour = landmarks.extract(('marker 0', 'marker 2', 'marker 15', 'marker 17'))
mesh_clip_ls = clip_with_contour(mesh_ls, start_time=0, fps=12, contour=contour, clip_bound='y', margin=0.5)

# Registration Approach Selection

In [ ]:
import time
start_time = time.time()

---

_P.S. Only activate one of the following code blocks in this section_

In [ ]:
from mesh4d.regist import reg_rbf

export_folder = 'output/rbf/'

o3_ls = obj3d.init_obj_series(
    mesh_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

o4 = reg_rbf.Obj4d_RBF(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=True,
    )

o4.add_obj(*o3_ls, landmarks=landmarks)
o4.load_markerset('landmarks', landmarks)
o4.regist('landmarks')

In [ ]:
from mesh4d.regist import reg_ecpd

export_folder = 'output/epcd/'

o3_ls = obj3d.init_obj_series(
    mesh_clip_ls,
    obj_type=obj3d.Obj3d_Deform
    )

o4 = reg_ecpd.Obj4d_ECPD(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=True,
    regist_points_num=1000,
    )

o4.add_obj(*o3_ls, landmarks=landmarks)
o4.load_markerset('landmarks', landmarks)
o4.regist('landmarks')

---

Export time duration

In [ ]:
duration = time.time() - start_time
print("computation time: {}".format(duration))

if is_export:
    utils.save_pkl_object(duration, export_folder, 'duration')

# Ultra-dense Motion Capture

## Control Landmarks

In [ ]:
kps_source = landmarks.get_time_coord(0)
o4.vkps_track(kps_source, frame_id=0)

In [ ]:
from mesh4d import kps
vkps = o4.assemble_markerset(name='vkps')
diff = kps.MarkerSet.diff(vkps, landmarks)

"""
inver_markers = ('marker 1', 'marker 4', 'marker 5', 'marker 6', 'marker 7', 'marker 8', 'marker 9', 'marker 10', 'marker 11', 'marker 12', 'marker 13')
diff = kps.MarkerSet.diff(vkps.extract(inver_markers), landmarks.extract(inver_markers))
"""

In [ ]:
if is_export:
    utils.save_pkl_object(diff, export_folder, 'diff')

In [ ]:
if is_plot:
    o4.export_deform_gif(export_folder, filename='vkps', kps_names=('vkps', 'landmarks'))

## Randomly Selected Landmarks

In [ ]:
from mesh4d import utils

random_landmarks = utils.load_pkl_object(test_landmarks_path)
random_kps = random_landmarks.get_frame_coord(0)

In [ ]:
o4.vkps_track(random_kps, frame_id=0, name='vkps_random')
vkps_random = o4.assemble_markerset(name='vkps_random')
vkps_random.get_trace_length()[2]

In [ ]:
if is_plot:
    o4.export_deform_gif(output_folder=export_folder, filename='vkps_random', kps_names=('vkps_random',))

## Full-field Deformation Estimation

In [ ]:
full_kps = o3_clip_ls[0].get_vertices()
o4.vkps_track(full_kps, frame_id=0, name='vkps_full')

In [ ]:
vkps_full = o4.assemble_markerset(name='vkps_full')
vkps_full.interp_field()

In [ ]:
from mesh4d.analyse import visual

_, starts, traces = vkps_full.get_trace_length()

if is_plot:
    visual.show_mesh_value_mask(starts, traces, min_threshold=500, max_threshold=900)

# K-fold cross-verfication

import time
from mesh4d import kps

import mesh4d
mesh4d.output_msg = False

for name in landmarks.markers.keys():
    # split dataset
    landmarks_test, landmarks_train = landmarks.split((name, ))

    start_time = time.time()

    o4 = reg_rbf.Obj4d_RBF(
        fps=120 / stride,
        enable_rigid=False,
        enable_nonrigid=True,
    )
    o4.add_obj(*o3_ls, landmarks=landmarks_train)
    o4.load_markerset('landmarks_train', landmarks_train)
    o4.load_markerset('landmarks_test', landmarks_test)

    duration = time.time() - start_time
    print("computation time: {}".format(duration))
    utils.save_pkl_object(duration, 'output/rbf/{}/duration.pkl'.format(name))

    # virtual key points tracking
    kps_source = landmarks_test.get_time_coord(0)
    o4.vkps_track(kps_source, frame_id=0, name='vkps_test')
    kps_source = landmarks_train.get_time_coord(0)
    o4.vkps_track(kps_source, frame_id=0, name='vkps_train')

    # quantitative estimation
    if is_plot:
        o4.show_deform_gif(output_folder='output/rbf/{}'.format(name), filename='vkps_train', kps_names=('vkps_train', 'landmarks_train'))
        o4.show_deform_gif(output_folder='output/rbf/{}'.format(name), filename='vkps_test', kps_names=('vkps_test', 'landmarks_test'))
        
    # quantitative estimation
    vkps = o4.assemble_markerset(name='vkps_train')
    diff_train = kps.MarkerSet.diff(vkps, landmarks_train)
    utils.save_pkl_object(diff_train, 'output/rbf/{}/diff_train.pkl'.format(name))

    vkps = o4.assemble_markerset(name='vkps_test')
    diff_test = kps.MarkerSet.diff(vkps, landmarks_test)
    utils.save_pkl_object(diff_test, 'output/rbf/{}/diff_test.pkl'.format(name))

    print("{} train error: {} test_error: {}".format(name, diff_train['diff_str'], diff_test['diff_str']))